In [13]:
from datasets import load_dataset
import sentencepiece as spm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
hello there

## Loading the Datasets

In [2]:
# Load the dataset
dataset = load_dataset("t1annnnn/Chinese_sentimentAnalyze")
train_dataset = dataset["train"]
test_dataset = dataset["test"]

## Tokenizing and Vectorizing

In [3]:
# Create and write to chinese_data.txt for SnetencePiece training
with open("chinese_data.txt", "w", encoding="utf-8") as file:
    for example in train_dataset:
        file.write(example["text"] + "\n")

#Training the data set model:
spm.SentencePieceTrainer.Train('--input=chinese_data.txt --model_prefix=chinese_model --vocab_size=8000 --character_coverage=0.9995 --model_type=bpe')

# Load the trained SentencePiece model
sp = spm.SentencePieceProcessor()
sp.Load("chinese_model.model")

# Tokenize all texts in the training dataset
tokenized_texts = [sp.EncodeAsPieces(text) for text in train_dataset['text']]
#for testing
tokenized_test_texts = [sp.EncodeAsPieces(text) for text in test_dataset['text']]

# Converting token lists back to strings
joined_texts = [' '.join(tokens) for tokens in tokenized_texts]
# Joining the tokens into strings for each document in the test set
joined_test_texts = [' '.join(tokens) for tokens in tokenized_test_texts]

# Creating a TfidfVectorizer object
vectorizer = TfidfVectorizer()

## Creating Feature variables

In [4]:
# Fitting the model and transforming the text data into TF-IDF vectors
X_train = vectorizer.fit_transform(joined_texts)
X_test = vectorizer.transform(joined_test_texts)

In [5]:
'''

# Calculate the sum of TF-IDF scores for each feature (word)
sum_tfidf = np.array(X.sum(axis=0)).flatten()
#print(sum_tfidf)
# Calculate the average by dividing by the number of documents
average_tfidf = sum_tfidf / X.shape[0]

# Get feature names to map indices to words
feature_names = vectorizer.get_feature_names_out()

# Map from feature name to average tf-idf score
word_to_avg_score = {feature_names[i]: average_tfidf[i] for i in range(len(feature_names))}
'''

'\n\n# Calculate the sum of TF-IDF scores for each feature (word)\nsum_tfidf = np.array(X.sum(axis=0)).flatten()\n#print(sum_tfidf)\n# Calculate the average by dividing by the number of documents\naverage_tfidf = sum_tfidf / X.shape[0]\n\n# Get feature names to map indices to words\nfeature_names = vectorizer.get_feature_names_out()\n\n# Map from feature name to average tf-idf score\nword_to_avg_score = {feature_names[i]: average_tfidf[i] for i in range(len(feature_names))}\n'

## Creating Target Variables

In [7]:
y_train = [data['label'] for data in train_dataset]
y_test = [data['label'] for data in test_dataset]

## Creating and Testing LogisticRegression model

In [8]:
from sklearn.linear_model import LogisticRegression

logistic_regression_model = LogisticRegression(max_iter=1000)
logistic_regression_model.fit(X_train, y_train)

y_pred = logistic_regression_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

print(classification_report(y_test, y_pred))

Accuracy: 0.7641297805985665
              precision    recall  f1-score   support

           0       0.75      0.79      0.77      9089
           1       0.78      0.74      0.76      9188

    accuracy                           0.76     18277
   macro avg       0.76      0.76      0.76     18277
weighted avg       0.76      0.76      0.76     18277



In [9]:
'''svm_classifier = SVC(kernel='linear') 
svm_classifier.fit(X_train, y_train)

y_pred = svm_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

print(classification_report(y_test, y_pred))'''

linear_svc_model = LinearSVC(max_iter=1000) 
linear_svc_model.fit(X_train, y_train)
y_pred = linear_svc_model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

C:\Users\Dell 3520\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


Accuracy: 0.7617223833233026
              precision    recall  f1-score   support

           0       0.75      0.78      0.76      9089
           1       0.77      0.74      0.76      9188

    accuracy                           0.76     18277
   macro avg       0.76      0.76      0.76     18277
weighted avg       0.76      0.76      0.76     18277



## Naive Bayes

In [12]:
naive_bayes_model = MultinomialNB()
naive_bayes_model.fit(X_train, y_train)
y_pred_NB = naive_bayes_model.predict(X_test)
from sklearn.metrics import accuracy_score, classification_report

# Calculate accuracy
print("Accuracy:", accuracy_score(y_test, y_pred_NB))

print(classification_report(y_test, y_pred_NB))


Accuracy: 0.7607375389834218
              precision    recall  f1-score   support

           0       0.76      0.76      0.76      9089
           1       0.76      0.76      0.76      9188

    accuracy                           0.76     18277
   macro avg       0.76      0.76      0.76     18277
weighted avg       0.76      0.76      0.76     18277



## Random Forest

In [14]:
random_forest_model = RandomForestClassifier(n_estimators=100, random_state=42)  
random_forest_model.fit(X_train, y_train)
y_pred_RF = random_forest_model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

print(classification_report(y_test, y_pred))

Accuracy: 0.7607375389834218
              precision    recall  f1-score   support

           0       0.76      0.76      0.76      9089
           1       0.76      0.76      0.76      9188

    accuracy                           0.76     18277
   macro avg       0.76      0.76      0.76     18277
weighted avg       0.76      0.76      0.76     18277

